In [1]:
cd ~/src/Classes/Ansible/

/Users/rick446/src/Classes/Ansible


# Managing Docker via Ansible

Ansible includes several modules for running container commands on a host:

- `docker_container` - manage docker containers
- `docker_image` - Manage docker images.
- `docker_image_facts` - Inspect docker images
- `docker_login` - Log into a Docker registry.
- `docker_network` - Manage Docker networks
- `docker_secret` - Manage docker secrets.
- `docker_service` - Manage docker services and containers.
- `docker_volume` - Manage Docker volumes

## Bootstrapping Docker

Hosts require a few things to run the Ansible Docker modules. In particular, hosts must have:

- Docker installed
- docker-py installed

To get these dependencies installed, we can use a playbook:

In [2]:
cat playbooks/bootstrap-docker.yaml

- hosts: db
  gather_facts: no
  become: yes
  tasks:
  - name: Install Docker
    apt:
      name: docker.io
      state: installed
  - name: start docker
    service:
      name: docker
      state: started
  - name: Adding user {{ ansible_ssh_user }} as a docker user
    user:
      name: '{{ansible_ssh_user}}'
      groups: docker
      append: yes
  - name: Install pip
    apt:
      name: python-pip
      state: installed
  - name: Add docker dependencies
    pip:
      name: docker-py
      state: latest


In [4]:
%%bash
ansible-playbook playbooks/bootstrap-docker.yaml


PLAY [db] **********************************************************************

TASK [Install Docker] **********************************************************
ok: [arborian-03.class.arborian.com]
ok: [arborian-04.class.arborian.com]

TASK [start docker] ************************************************************
ok: [arborian-03.class.arborian.com]
ok: [arborian-04.class.arborian.com]

TASK [Adding user {{ ansible_ssh_user }} as a docker user] *********************
ok: [arborian-03.class.arborian.com]
ok: [arborian-04.class.arborian.com]

TASK [Install pip] *************************************************************
ok: [arborian-03.class.arborian.com]
ok: [arborian-04.class.arborian.com]

TASK [Add docker dependencies] *************************************************
ok: [arborian-03.class.arborian.com]
ok: [arborian-04.class.arborian.com]

PLAY RECAP *********************************************************************
arborian-03.class.arborian.com : ok=5    changed=0    un

Now that that's done, we can run playbooks with our `docker_container` module, or just invoke it as a one-off:

In [6]:
%%bash
ansible db -m docker_container -a 'name=redis image=redis:latest state=started'

arborian-04.class.arborian.com | SUCCESS => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "",
            "Args": [
                "redis-server"
            ],
            "Config": {
                "AttachStderr": false,
                "AttachStdin": false,
                "AttachStdout": false,
                "Cmd": [
                    "redis-server"
                ],
                "Domainname": "",
                "Entrypoint": [
                    "docker-entrypoint.sh"
                ],
                "Env": [
                    "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
                    "GOSU_VERSION=1.10",
                    "REDIS_VERSION=4.0.2",
                    "REDIS_DOWNLOAD_URL=http://download.redis.io/releases/redis-4.0.2.tar.gz",
                    "REDIS_DOWNLOAD_SHA=b1a0915dbc91b979d06df1977fe594c3fa9b189f1f3d38743a2948c9f7634813"
                ],
                "ExposedPorts": {

# Lab: Starting and Stopping Containers

Start (and then remove) a few containers on the `db` servers. Examine the output of the `docker_container` command.

You can find many containers to start at https://hub.docker.com/explore/

# Managing Containers using Ansible-Container

Where the Ansible docker_* modules allow you to manage Docker via command-line tools, the Ansible-Container project provides for the management and orchestration of containers via Ansible playbooks. (The container becomes the managed host).

## Creating an ansible-container project

To get started, we use the `ansible-container init` command.

### Create an 'empty' project

`$ ansible-container init` with no arguments will create a small set of files in the current directory with minimal configuration. I did this in the simple_container directory:

In [13]:
ls simple-container/

ansible-deployment/       ansible.cfg               meta.yml
ansible-requirements.txt  container.yml             requirements.yml


To actually build our set of containers, we use the `build` and `run` commands:

In [14]:
%%bash
cd simple-container/
ansible-container build
ansible-container run

Building Docker Engine context...	
Starting Docker build of Ansible Container Conductor image (please be patient)...	
Parsing conductor CLI args.
Docker™ daemon integration engine loaded. Build starting.	project=simple-container
All images successfully built.
Conductor terminated. Cleaning up.	command_rc=0 conductor_id=9312ab758b6862b06ca196675db5f253932bdd7a6c8e10038d2e98ce80b9d95b save_container=False
Parsing conductor CLI args.
Engine integration loaded. Preparing run.	engine=Docker™ daemon
 [WARNING]: Could not match supplied host pattern, ignoring: all
 [WARNING]: provided hosts list is empty, only localhost is available

PLAY [Deploy simple-container] *************************************************

TASK [docker_service] **********************************************************
ok: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=1    changed=0    unreachable=0    failed=0

All services running.	playb

## Starting from the galaxy

We can also start from a project on Ansible Galaxy by adding its name to our `ansible-container init` command. (I did this in the `container` directory:

`ansible-container init ansible.django-template`

## Lab: Installing and using Ansible-Container

- Install Ansible-Container (or use the playbook playbooks/prepare-ansible-container.yaml)
- Install and run the ansible.django-template app in the `container` directory
- Change the database password in container.yml and rebuild and rerun the container

# Some notes

`ansible-container build` creates a **Conductor Container** which is responsible for doing all the actual ansible work. 

The *conductor* will build the various containers ("Services") in your `container.yml` file. Each Ansible role for a service becomes a *layer* in the generated image.

You can install various Ansible requirements in the *conductor* by adding them to the ansible-requirements.txt file

There's much more, including deployment to Docker Swarm or K8S. See http://docs.ansible.com/ansible-container/ for more information.